In [ ]:
!pip install jupyterlab ipykernel ipywidgets datasets  torch torchvision torchaudio transformers peft evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 91.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 60.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 36.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 59.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69

In [ ]:
# AI Debate Partner - Argument Generation (Fixed Training)
# Fixes ValueError: The model did not return a loss

## Step 1: Import Libraries
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling  # Changed from DataCollatorWithPadding
)
from peft import LoraConfig, get_peft_model
from datasets import Dataset
import pandas as pd
import torch

## Step 2: Load and Preprocess Dataset
# Load CSV file
df = pd.read_csv('/content/train.csv')

# Preprocessing
df = df[['topic', 'argument', 'stance_WA']].dropna()

# Convert stance to readable labels
df['stance_label'] = df['stance_WA'].map({1: "pro", -1: "con"})

# Create prompt column
df['text'] = "Topic: " + df['topic'] + "; Stance: " + df['stance_label'] + "; Argument: " + df['argument'] + "<|endoftext|>"

# Create Dataset object
dataset = Dataset.from_pandas(df[['text']])

## Step 3: Tokenization
model_checkpoint = 'gpt2-medium'
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
tokenizer.pad_token = tokenizer.eos_token

def tokenize_function(examples):
    # Tokenize entire text sequence
    return tokenizer(
        examples['text'],
        truncation=True,
        max_length=256,
        padding="max_length"
    )

tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=['text'])

## Step 4: Model Setup
model = AutoModelForCausalLM.from_pretrained(model_checkpoint)

## Step 5: PEFT Configuration
peft_config = LoraConfig(
    r=8,
    lora_alpha=32,
    lora_dropout=0.05,
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

## Step 6: Training Setup (FIXED)
# Use language modeling data collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # For causal language modeling
)

training_args = TrainingArguments(
    output_dir="ai-debate-generator",
    learning_rate=2e-4,
    per_device_train_batch_size=4,
    num_train_epochs=3,  # Reduced for faster iteration
    weight_decay=0.01,
    logging_steps=100,
    save_steps=500,
    report_to="none",
    prediction_loss_only=True,  # Ensure loss is computed
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator,  # Critical fix
)

## Step 7: Training
trainer.train()  # Should now compute loss

## Step 8: Argument Generation Function
def generate_argument(topic: str, stance: str):
    stance = stance.lower()
    if stance not in ["pro", "con"]:
        raise ValueError("Stance must be 'pro' or 'con'")

    prompt = f"Topic: {topic}; Stance: {stance}; Argument:"
    inputs = tokenizer(prompt, return_tensors="pt")

    # Generate with attention to context length
    outputs = model.generate(
        input_ids=inputs.input_ids,
        attention_mask=inputs.attention_mask,
        max_new_tokens=100,  # Limit output length
        temperature=0.7,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id  # Ensure proper termination
    )

    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    argument = generated_text.split("Argument:")[1].strip()
    return argument

## Step 9: Interactive Usage

    user_topic = input("Enter debate topic: ")
    user_stance = input("Enter your stance (pro/con): ")
    argument = generate_argument(user_topic, user_stance)
    print("\nGenerated Argument:")
    print("-------------------")
    print(argument)

## Step 10: Save Model
model.save_pretrained("ai-debate-generator-model")
tokenizer.save_pretrained("ai-debate-generator-tokenizer")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

trainable params: 786,432 || all params: 355,609,600 || trainable%: 0.2212


No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
100,2.682700
200,2.008300
300,1.985200
400,1.927200
500,1.832100
600,1.812800
700,1.777300
800,1.741900
900,1.744700
1000,1.762100


Step,Training Loss
100,2.682700
200,2.008300
300,1.985200
400,1.927200
500,1.832100
600,1.812800
700,1.777300
800,1.741900
900,1.744700
1000,1.762100


('ai-debate-generator-tokenizer/tokenizer_config.json',
 'ai-debate-generator-tokenizer/special_tokens_map.json',
 'ai-debate-generator-tokenizer/vocab.json',
 'ai-debate-generator-tokenizer/merges.txt',
 'ai-debate-generator-tokenizer/added_tokens.json',
 'ai-debate-generator-tokenizer/tokenizer.json')

In [5]:
import shutil

# Zip the folder (e.g., "my_model") into "my_model.zip"
shutil.make_archive("my_model", 'zip', "/content/ai-debate-generator-model")


'/content/my_model.zip'

In [6]:


# Zip the folder (e.g., "my_model") into "my_model.zip"
shutil.make_archive("tokenizer", 'zip', "/content/ai-debate-generator-tokenizer")


'/content/tokenizer.zip'